# gurobiによる制約最適化を行う

In [144]:
import math
import random
import networkx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import networkx
from gurobipy import *

## 変数
- $x_{ik}$ $x_{jl}$ : 人iが避難施設kに避難した場合に１　バイナリー変数

## 定数

- $D_{kl}$ : 避難施設kとlの間に定義される伝染危険率
    - 二つの距離が近いと大きい

- $W_{ij} $ : 人i,jでの伝染危険率(以下の二つの重み付き自乗和)

    - 感染の差の自乗とmax(jの感染率 – iの感染率,0)・iの死亡率 + max(iの感染率 – jの感染率,0)・ jの死亡率
    - 感染率の差の自乗和

## 目的関数
-  min $\sum_{ijkl}^{} W_{ij} D_{kl}  x_{ik}  x_{jl}$


## 制約
- すべての人 iに対して，x[i,k]の施設kに対する合計 = 1   

- 施設kに対してx[i]=kの人に対する合計が施設kの容量以下

## 必要な集合
- 人の集合 people
- 人の感染率の集合 infec_rate
- 人の死亡率の集合 mort_rate
- 施設集合 shelters
- 施設の位置情報 locations
- 施設の容量 capacity

# random.seed(0)

In [146]:
N = 10 #人の数
people = [i for i in range(N)] #人の集合
infec_rate,mort_rate = {},{}
for i in people:
    infec_rate[i] = random.randint(0,100)/100
    mort_rate[i] = random.randint(0,100)/100
print(infec_rate)
print(mort_rate)

{0: 0.49, 1: 0.53, 2: 0.33, 3: 0.62, 4: 1.0, 5: 0.61, 6: 0.74, 7: 0.64, 8: 0.36, 9: 0.96}
{0: 0.97, 1: 0.05, 2: 0.65, 3: 0.51, 4: 0.38, 5: 0.45, 6: 0.27, 7: 0.17, 8: 0.17, 9: 0.12}


In [147]:
N_s = 5
shelters = [i for i in range(N_s)] #施設の集合
capacity = {}
for i in shelters:
    capacity[i] = random.randint(0,5)
locations= np.random.rand(N_s, 2) * 100
print(locations)
print(capacity)

[[22.62105182 11.31026766]
 [58.41387508 54.1486509 ]
 [90.50280073 41.65557359]
 [32.26651766 50.39966179]
 [ 2.26331577 69.26583333]]
{0: 4, 1: 2, 2: 4, 3: 5, 4: 4}


In [148]:
#D[k,l]施設間の感染率　簡易的に距離で置いている
x_y = np.array(locations)
x = x_y[:, 0]
y = x_y[:, 1]
D = np.sqrt((x[:, np.newaxis] - x[np.newaxis, :]) ** 2 +(y[:, np.newaxis] - y[np.newaxis, :]) ** 2)
D

array([[ 0.        , 55.82341153, 74.35569532, 40.26183981, 61.42707066],
       [55.82341153,  0.        , 34.43510026, 26.41475382, 58.14993134],
       [74.35569532, 34.43510026,  0.        , 58.88908001, 92.45827788],
       [40.26183981, 26.41475382, 58.88908001,  0.        , 35.44184747],
       [61.42707066, 58.14993134, 92.45827788, 35.44184747,  0.        ]])

In [149]:
#W[i,j]
W = np.zeros((N,N))
for i in people:
    for j in people:
        w1 = max(infec_rate[j] - infec_rate[i],0) * mort_rate[i] + max(infec_rate[i] - infec_rate[j],0) * mort_rate[j]
        w2 = (infec_rate[j] - infec_rate[i]) ** 2
        W[i,j] = w1*w2
W

array([[0.0000000e+00, 6.2080000e-05, 2.6624000e-03, 2.1310900e-03,
        1.2867147e-01, 1.6761600e-03, 1.5156250e-02, 3.2737500e-03,
        3.7349000e-04, 1.0070831e-01],
       [6.2080000e-05, 0.0000000e+00, 5.2000000e-03, 3.6450000e-05,
        5.1911500e-03, 2.5600000e-05, 4.6305000e-04, 6.6550000e-05,
        8.3521000e-04, 3.9753500e-03],
       [2.6624000e-03, 5.2000000e-03, 0.0000000e+00, 1.5852850e-02,
        1.9549595e-01, 1.4268800e-02, 4.4798650e-02, 1.9364150e-02,
        1.7550000e-05, 1.6253055e-01],
       [2.1310900e-03, 3.6450000e-05, 1.5852850e-02, 0.0000000e+00,
        2.7984720e-02, 4.5000000e-07, 8.8128000e-04, 4.0800000e-06,
        2.9879200e-03, 2.0045040e-02],
       [1.2867147e-01, 5.1911500e-03, 1.9549595e-01, 2.7984720e-02,
        0.0000000e+00, 2.6693550e-02, 4.7455200e-03, 7.9315200e-03,
        4.4564480e-02, 7.6800000e-06],
       [1.6761600e-03, 2.5600000e-05, 1.4268800e-02, 4.5000000e-07,
        2.6693550e-02, 0.0000000e+00, 9.8865000e-04, 1.21

In [150]:
model = Model()

In [151]:
#人iが避難施設kに避難した場合に1になる0-1変数 x[i,k] x[j,l]
x = {}
for i in people:
    for k in shelters:
        x[i,k] = model.addVar(ub=1, vtype="B", name="x(%s,%s)"%(i,k))
        
for j in people:
    for l in shelters:
        x[j,l] = model.addVar(ub=1, vtype="B", name="x(%s,%s)"%(j,l))
        
model.update()

In [152]:
# z[i,k,j,l]という新しい変数をおく
z = {}
for i in people:
    for k in shelters:
        for j in people:
            for l in shelters:
                z[i,k,j,l] = model.addVar(vtype="B",name="z(%s,%s,%s,%s)"%(i,k,j,l))
model.update()

## 制約

In [153]:
# すべての人 iに対して，x[i,k]の施設kに対する合計 = 1   
for i in people:
    model.addConstr(quicksum(x[i,k] for k in shelters)== 1)
    
#すべての施設kに対して，x[i,k] の人に対する合計が施設kの容量以下
for k in shelters:
    model.addConstr(quicksum(x[i,k] for i in people) <= capacity[k])
model.update()

In [154]:
# Z[i,k,j,l]の制約
for i in people:
    for k in shelters:
        for j in people:
            for l in shelters:
                model.addConstr(z[i,k,j,l] <= x[i,j] + x[j,l] -1)

KeyError: (0, 5)

In [155]:
#目的関数
model.setObjective(quicksum(W[i,j]*D[k,l]*z[i,k,j,l]
                            for i in people
                            for j in people
                            for k in shelters
                            for l in shelters
                            if j>i), GRB.MINIMIZE)

model.update()